# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [4]:
# imports
import os
import json
from dotenv import load_dotenv
from scraper import fetch_website_contents, fetch_website_links
from openai import OpenAI
from IPython.display import Markdown, display, update_display




So let's look at what this week one exercise is.

I want to see your familiarity with OpenAI and Olama to build a tool that takes a technical question,

which you can just type right here and responds with an explanation of that question.

So I've got a little couple of hints down below, but use this as a way to call a model so that you

could retype a question in here, and the model will explain it.

In [8]:
# constants

MODEL_GPT = 'openai/gpt-4o-mini'
MODEL_LLAMA = 'meta-llama/llama-3.3-70b-instruct:free'



In [9]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
elif api_key and api_key.startswith('sk-or-v1') and len(api_key)>10:
    print("You are bang on...dude , because you're using OpenRouter!\n") 
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

You are bang on...dude , because you're using OpenRouter!



In [10]:


openai = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv('OPENROUTER_API_KEY'), 
)



In [14]:
# here is the question; type over this to ask something new

question = """
You are a technical assistant, and you are extremely good in python. 
You will take a technical question and respond with an explanation of that question. 
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
messages = [{"role": "user", "content": question}]




#  Streaming `without` using display and Markdown class from IPython.display 

In [ ]:
# Get gpt-4o-mini to answer, with streaming
# Streaming with out using display and Markdown class from IPython.display 
def call_gpt_4o():
    response = []
    stream = openai.chat.completions.create( model= MODEL_GPT,messages= messages , stream=True) 
    for chunk in stream:
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end="", flush=True)
            response.append(delta.content) 
    return response 
    

print(call_gpt_4o())

The code snippet you provided uses the `yield from` statement in Python along with a set comprehension. Let's break down what it does step-by-step.

### Components of the Code:

1. **Set Comprehension**: 
   - `{book.get("author") for book in books if book.get("author")}`: This part of the code creates a set from the `books` iterable.
   - It iterates over each `book` in the `books` collection.
   - For each `book`, it tries to retrieve the value associated with the key `"author"` using the `.get()` method. If the `"author"` key exists and the value is truthy (in this case, not `None` or an empty string), it adds that author to the set.
   - The use of a set ensures that only unique authors are collected, eliminating any duplicates.

2. **`yield from`**:
   - The `yield from` statement is used in a generator function to yield all values from another iterable (in this case, the set created by the comprehension).
   - This means that the generator will yield each unique author one by one

#  Streaming `WITH` using display and Markdown class from IPython.display 

In [25]:
def call_gpt_4o():
    response = ''
    stream = openai.chat.completions.create( model= MODEL_GPT,messages= messages , stream=True) 
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id) 
      
    

print(call_gpt_4o())

The line of code you've presented uses the `yield from` statement in combination with a set comprehension. Let's break it down step by step.

### Explanation of Components:

1. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. 
   - It iterates over a collection `books` (presumably a list or another iterable of dictionaries).
   - For each `book` in `books`, it attempts to retrieve the value associated with the key `"author"` using the `get` method.
   - The `if book.get("author")` part is a condition that filters out books that do not have an author. In other words, it only includes `book.get("author")` in the resulting set if the value is not `None` (or an empty string, depending on context).
   - The comprehension creates a set of unique author names extracted from the `books` collection.

2. **Yield From**:
   - The `yield from` statement is used in generator functions to yield all values from an iterable (in this case, the set created by the comprehension) one by one.
   - When you use `yield from <iterable>`, you're essentially allowing the generator to yield each item in the iterable in turn, pausing and resuming the generator's state as necessary.

### What Does the Code Do?

- This line constructs a set of unique authors from a collection of books and yields each author one at a time from a generator function.
- If `books` contains dictionaries where some may not have an `"author"` key or where the value is `None`, those entries will be filtered out. 

### Why Use This Approach?

1. **Efficiency**: By using a set, it inherently ensures that there are no duplicate authors. This is useful if you want to work with a list of unique authors without manually checking for duplicates.
2. **Generator Behavior**: Using `yield from` allows the function that contains this code to produce results incrementally (as a generator), which can be memory-efficient and suitable for handling large datasets, as it does not require all unique authors to be stored in memory at once.
3. **Readability**: This one-liner approach is compact and expressive, making it clear that the intent is to extract and yield unique authors.

### Example:

Assuming `books` is defined as follows:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
]
```

The code will yield:
- "Author A"
- "Author B"

Only these two unique authors will be yielded because "Author A" appears twice, and the book with no author is excluded.

In summary, this code neatly extracts and yields unique authors from a collection of book entries while filtering out any entries without a specified author.

None


In [ ]:
def call_gpt_4o():
    response = ''
    stream = openai.chat.completions.create( model= MODEL_GPT,messages= messages , stream=True) 
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id) 
      
    

print(call_gpt_4o())

In [32]:
# Get Llama 3.2 to answer

def call_llama_3_3():
    #   
    try:
        response = ''
        stream = openai.chat.completions.create(model= MODEL_LLAMA,messages= messages , stream=True)
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            update_display(Markdown(response), display_id=display_handle.display_id) 
    except Exception as e:
        print(f"Error: {e}") 
    
    
print(call_llama_3_3()) 

Error: Error code: 429 - {'error': {'message': 'Provider returned error', 'code': 429, 'metadata': {'raw': 'meta-llama/llama-3.3-70b-instruct:free is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations', 'provider_name': 'Venice', 'is_byok': False}}, 'user_id': 'user_3A4ekcvPJiShYdwdz2iL4cUskqW'}
None
